In [ ]:
from sklearn.datasets import fetch_20newsgroups
news_data=fetch_20newsgroups(subset='all', random_state=156)

print(news_data.keys())

In [ ]:
import pandas as pd

print('target 클래스의 값과 분포도\n', pd.Series(news_data.target).value_counts().sort_index())
print('target 클래스의 이름들 \n', news_data.target_names)

In [ ]:
print(news_data.data[0])

In [ ]:
from sklearn.datasets import fetch_20newsgroups

#subset='train'으로 학습용 데이터만 추출, remove=(headers','footers','quotes')로 내용만 추출
train_news=fetch_20newsgroups(subset='train',remove=('headers','fotters','quotes'),random_state=156)
X_train=train_news.data
y_train=train_news.target

#subset='test'으로 테스트 데이터만 추출, remove=['headers','footers','quotes')로 내용만 추출
test_news=fetch_20newsgroups(subset='test',remove=('hearders','footers','quotes'),random_state=156)
X_test=test_news.data
y_test=test_news.target
print('학습 데이터 크기 {0}, 텍스트 데이터 크기 {1}'.format(len(train_news.data), len(test_news.data)))

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# Count Vectorization으로 feature extraction 변환 수행. 
cnt_vect = CountVectorizer()

cnt_vect.fit(X_train)
X_train_cnt_vect = cnt_vect.transform(X_train)

# 학습 데이터로 fit( )된 CountVectorizer를 이용하여 테스트 데이터를 feature extraction 변환 수행. 
X_test_cnt_vect = cnt_vect.transform(X_test)

print('학습 데이터 Text의 CountVectorizer Shape:',X_train_cnt_vect.shape)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings('ignore')

#logisticRegression을 이용하여 학습/예측/평가 수행
lr_clf=LogisticRegression(solver='liblinear')
lr_clf.fit(X_train_cnt_vect,y_train)
pred=lr_clf.predict(X_test_cnt_vect)
print('CounterVectorized Logistic Regression의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test,pred)))

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

#TF-IDF 벡터화를 적용해 학습 데이터 세트와 테스트 데이터 세트 변환
tfidf_vect = TfidfVectorizer()
tfidf_vect.fit(X_train)
X_train_tfidf_vect=tfidf_vect.transform(X_train)
X_test_tfidf_vect=tfidf_vect.transform(X_test)

#LogisticRegression 을 이용해 학습/예측/평가 수행
lr_clf=LogisticRegression(solver='liblinear')
lr_clf.fit(X_train_tfidf_vect,y_train)
pred=lr_clf.predict(X_test_tfidf_vect)
print('TF-IDF Logistic Regression의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test,pred)))

In [ ]:
#stop words 필터링을 추가하고 ngram을 기본 (1,1)에서 (1,2)로 변경해 피처 벡터화 적용
tfidf_vect=TfidfVectorizer(stop_words='english',ngram_range=(1,2), max_df=300 )
tfidf_vect.fit(X_train)
X_train_tfidf_vect=tfidf_vect.transform(X_train)
X_test_tfidf_vect=tfidf_vect.transform(X_test)

lr_clf=LogisticRegression(solver='liblinear')
lr_clf.fit(X_train_tfidf_vect,y_train)
pred=lr_clf.predict(X_test_tfidf_vect)
print('TF-IDF Logistic Regression의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test,pred)))

In [ ]:
from sklearn.model_selection import GridSearchCV

#최적의 값 c값 도출 튜닝 수행, cv는 3폴드 세트로 설정
params={'C':[0.01,0.1,1,5,10]}
grid_cv_lr=GridSearchCV(lr_clf, param_grid=params, cv=3, scoring='accuracy', verbose=1)
grid_cv_lr.fit(X_train_tfidf_vect,y_train)

# 최적 C 값으로 학습된 grid_cv로 예측 수행하고 정확도 평가. 
print('Logistic Regression best C parameter:', grid_cv_lr.best_params_)
print('TF-IDF Logistic Regression의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test,pred)))

In [ ]:
from sklearn.pipeline import Pipeline

#TfidVectorizer 객체를 tfdif_vect로, LogisticRegression 객체를 lr_clf로 생성하는 pipeline 생성
pipeline=Pipeline([('tfidf_vect',TfidfVectorizer(stop_words='english',ngram_range=(1,2),max_df=300)),
                   ('lr_clf',LogisticRegression(solver='liblinear',C=10))])

#별도의 TfidVectorizer 객체의 fit(), transform()과 LogisticRegression의 fit(), predict()가 필요없음.
#pipeline의 fit()과 predict()만으로 한꺼번에 피처 벡터화와 ML학습/예측이 가능
pipeline.fit(X_train,y_train)
pred=pipeline.predict(X_test)
print('Pipeline을 통한 Logistic Regression의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test,pred)))

In [ ]:
from sklearn.pipeline import Pipeline

pipeline=Pipeline([('tfidf_vect',TfidfVectorizer(stop_words='english')),
                   ('lr_clf',LogisticRegression(random_state=156))])

#Pipeline에 기술된 각각의 객체 변수에 언더바 2개를 연달아 붙여 GridSearchCV에 사용될 파라미텨/하이퍼 파라미터 이름 값을 설정
params={'tfidf_vect__ngram_range':[(1,1),(1,2),(1,3)],
        'tfidf_vect__max_df':[100,300,700],
        'lr_clf__C':[1,5,10]}

#GridSearchCV의 생성자 안에 Estimator가 아닌 Pipeline 객체 입력
grid_cv_pipe=GridSearchCV(pipeline, param_grid=params, cv=3, scoring='accuracy', verbose=1)
grid_cv_pipe.fit(X_train,y_train)
print(grid_cv_pipe.best_params_, grid_cv_pipe.best_score_)

pred=grod_cv_pipe.predict(X_test)
print('Pipeline을 통한 Logistic Regression의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test,pred)))